### SQL for extracting items in order of appearance

In [ ]:
SELECT 
      ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS rank,
      ce.ITEMID,
      di.LABEL AS item_name,
      ce.VALUEUOM,
      COUNT(*) AS entry_count
  FROM read_csv_auto('norm_pneumonia.csv', header=true) AS ce
  JOIN read_csv_auto('D_ITEMS.csv', header=true) AS di
    ON ce.ITEMID = di.ITEMID
  GROUP BY ce.ITEMID, di.LABEL, ce.VALUEUOM
  ORDER BY entry_count DESC;


### SQL for sorting relevant (with CHARTEVENTS data) icu stays based on LOS

In [ ]:
SELECT
    icu.ICUSTAY_ID,
    icu.SUBJECT_ID,
    icu.HADM_ID,
    icu.INTIME,
    icu.OUTTIME,
    icu.LOS
FROM read_csv_auto('ICUSTAYS.csv', header=true) AS icu
JOIN read_csv_auto('DIAGNOSES_ICD.csv', header=true) AS diag
    ON icu.HADM_ID = diag.HADM_ID AND icu.SUBJECT_ID = diag.SUBJECT_ID
WHERE diag.ICD9_CODE = '48283'
    AND icu.ICUSTAY_ID IN (
    SELECT DISTINCT ce.ICUSTAY_ID
    FROM read_csv_auto('CHARTEVENTS.csv', header=true, ignore_errors=true) AS ce
    WHERE ce.ICUSTAY_ID IS NOT NULL
    )
ORDER BY icu.LOS ASC
